# Financial Algebra #

### My refresher notes on Algebra commonly used in Finance ###

\begin{align}
(AB)^{-1} = B^{-1}A^{-1}
\end{align}

A matrix that has an inverse is called *Non Singular Matrix*, otherwise its called 
*Singular Matrix*. A singular matrix has at least one row (or column) that has a linear 
relationship with another row (or column).


### Determinant of a matrix ###

$$det(A) = |A| = det
 \begin{bmatrix}
  a & b \\
  c & d \\  
 \end{bmatrix} = \mathbf{ad} - \mathbf{bc} $$
Determinant of a matrix is zero if the matrix is singular. A matrix with zero determinant has no inverse. To inverse a matrix, use determinant and cofactors as below
$$\mathbf{A}^{-1} = \begin{bmatrix}
  a & b \\
  c & d \\  
 \end{bmatrix}^{-1} = \frac{1}{\mid A \mid} = \frac{1}{\mathbf{ad} - \mathbf{bc}} \begin{bmatrix}
  d & -b \\
  -c & a \\  
 \end{bmatrix}$$